#  Model tuning and hyperparameter optimization

<i>Balázs Kégl (CNRS / PS-CDS) and Alex Gramfort (Inria)</i>

## Introduction
Virtually all learning models come with a set of parameters that should be tuned. To distinguish them from the parameters of the models set during training (learned parameters like neural weigths, tree cuts, or ensemble coefficients), we call them <i>hyperparameters</i>. Typical examples are the number of iterations and the tree depth in random forests, or the <code>C</code> or <code>gamma</code> of a support vector classifier. Scikit-learn has default values for all these parameters, but there is no guarantee that any of the classifiers perform optimally (or even close to optimally) using these default parameters. Worse: the optimal parameters vary depending on the data set, so tuning them is the task of the analyst, not the task of the provider of the library.

Hyperparameters come in all shapes and types. Some of them are crucial to be tuned (like <code>n_estimators</code> in ensemble methods), the default value of some others works fine most of the time (say, <code>min_samples_split</code> of a random forest). Scikit-learn also has auxiliary parameters that have no effects on the model (such as <code>verbose</code>), so knowing which parameters to tune requires understanding a little bit what the particular model is about. Most of the hyperparameters have typical ranges, although they can depend of course on the characteristics of the data set. There are even so called <i>conditional</i> hyperparameters whose <i>existence</i> depend on the value of some other hyperparameters: for example, when you add a new layer to a neural network, suddenly you have 3-5 new hyperparameters to tune.

Important hyperparameters can usually be interpreted as complexity parameters of the learned function. Sometimes the link is easy to see: an ensemble with a large number of deep trees have more degrees of freedom than a small ensemble of small trees. Sometimes the link between hyperparameters and complexity is more indirect, and understanding it requires some basic notion of <i>regularization</i>. One general notion that helps to understand the behavior of the learning algorithms with respect to their hyperparameters is <i>overfitting</i> and <i>underfitting</i> (this is why it is important to understand whether complexity increases or decreases as the hyperparameter grows). Let us call the model that we could obtain if we knew the distribution (or had infinite data) the <i>ideal</i> model. The goal of learning is to get as close to this model as possible, using only a finite training set. We may have two problems
<ol>
<li>
Underfitting occurs when our model is not flexible enough to pick up the details of the ideal function, so even with infinite data we could not get close to the ideal function. The difference between the error of the ideal function and the error of the best function we could pick from our model class (parametrized by the hyperparameters) is the <i>approximation</i> error. Underfitting means that the approximation error is larger than optimal. Increasing the complexity of the model class decreases the approximation error (independently of the data set).
<li>
Overfitting occurs when our model is too flexible so it learns the training data "by heart" and does not generalize well to unseen test data. The difference between the error of the function (learned on the data set) and the error of the best function we could pick from our model class (parametrized by the hyperparameters) is the <i>estimation</i> error. Overfitting means that estimation error is larger than optimal. Increasing the complexity of the model class (while keeping the data set fixed) increases the estimation error.
</ol>
Note the crucial difference between the two notions: the level of underfitting is independent of the data set; it only depends on the data distribution and the function set (parametrized by the hyperparameters), whereas the level of overfitting, of course, depends on the data set. Here are some general rules that could guide you in this landscape.
<ol>
<li> Overfitting increases as the model complexity grows.
<li> Overfitting increases as the data complexity (for example, the number of features) grows.
<li> Overfitting decreases as the data size grows.
</ol>

If we could compute the data distribution or had access to an infinite data set we could determine the estimation and approximation errors and set the model complexity to minimize their sum. Of course, this is impossible so these notions are not constructive (but can help you to understand the underlying phenomena). In practice, we <i>estimate</i> the <i>generalization</i> error (the sum of the approximation and estimation errors) on a held out (validation) set. The pragmatics of hyperparameter optimization are thus relatively simple: find the hyperparameters that minimize the validation error. The practical difficulties are the following:
<ol>
<li> Our estimate has a variance: it is based on a finite validation set. Optimizing a "noisy" function can be tricky. Using cross validation schemes makes this even worse: since training and test sets overlap, complex correlation structures are introduced into the error estimates.
<li> Evaluating the validation error at a vector of hyperparameter values requires training a model which may be slow. 
<li> The number of hyperparameter combinations increases exponentially with the number of hyperparameters.
</ol> 

The first step of hyperparameter tuning is usually trial and error. Try some combinations and look at how the error behaves. Once you have a "feeling" about what range should be explored and the sensitivity of the error to the hyperparameters, you can do a grid search (using either <code>GridSearchCV</code> from scikit learn, or by coding it). In this notebook we coded a simple grid search so you understand how it works. Once the number of hyperparameters is large (say, larger than 2) grid search will become increasingly time consuming. The main inefficency is a result of it not being intelligent: it will exhaustively look at <i>all</i> combinations, even those that are "trivially" bad. 

The main principle to improve on grid search is smoothness in the hyperparameter space: we assume that models with similar hyperparameters are not very different. This allows sophisticated automatic methods to direct the search toward promising regions while also keeping exploring unexplored regions. Most of these methods are based on <a href="http://en.wikipedia.org/wiki/Bayesian_optimization">Bayesian optimization</a>. We will work with the <a href="https://github.com/hyperopt/hyperopt/wiki/FMin">hyperopt</a> package. It can be installed with:

<code>pip install hyperopt</code>

In [ ]:
# !pip install hyperopt

## Import some libraries

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

## Fetch the data and load it in pandas, and convert it into numpy
We can use any data set here. By the end of this section, two numpy arrays, <code>X</code> and <code>y</code> shoud be created.

In [ ]:
data = pd.read_csv("train.csv", index_col=0)
data['real_period'] = data['period'] / data['div_period']
data.head(3)

In [ ]:
X = data[['magnitude_b', 'magnitude_r', 'real_period']].values
y = data['type'].values

In [ ]:
X.shape, y.shape, X.dtype, y.dtype

We're good !

## Train a classifier

without worrying too much about hyperparameters

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold

clf = RandomForestClassifier(n_estimators=100, max_depth=8)
cv = KFold(n_splits=5, random_state=42)
scores = cross_val_score(clf, X_train, y_train, cv=cv, scoring='accuracy')
print("Accuracy: {:.4f} +/-{:.4f}".format(np.mean(scores), np.std(scores)))

### Train a classifier using a grid of hyperparameters

In [ ]:
n_estimators_list = [2, 3, 5, 10, 20, 30, 50, 100, 200, 500]
max_depth_list = [1, 2, 3, 5, 7, 10, 15, 20, 30]
accuracies = {}
accuracy_stds = {}

for n_estimators in n_estimators_list:
    print(n_estimators)
    for max_depth in max_depth_list:
        clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, n_jobs=1)
        scores = cross_val_score(clf, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=3)
        accuracies[(n_estimators, max_depth)] = np.mean(scores)
        accuracy_stds[(n_estimators, max_depth)] = np.std(scores)

In [ ]:
accuracies

In [ ]:
np.max(list(accuracies.values()))

Now let's do it using scikit-learn <a href="http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html">GridSearchCV</a> object.

In [ ]:
from sklearn.model_selection import GridSearchCV

tuned_parameters = {'n_estimators': n_estimators_list, 'max_depth': max_depth_list}
clf = RandomForestClassifier(random_state=42)

gs = GridSearchCV(clf, tuned_parameters, cv=cv, scoring='accuracy', n_jobs=3)
gs.fit(X, y)

In [ ]:
print("Best parameters set found on development set:")
print(gs.best_params_)
print()
print("Grid scores on training set:")
print()
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
cv_results = pd.DataFrame(gs.cv_results_)
cv_results.head(5)

In [ ]:
cv_results[['mean_test_score', 'param_max_depth', 'param_n_estimators']].head(15)

In [ ]:
cv_results[['mean_test_score', 'param_max_depth', 'param_n_estimators']].sort_values(by='mean_test_score', ascending=False).head()

### Plot some learning curves

In [ ]:
this_cv_results = cv_results[cv_results['param_n_estimators'] == 10]
plt.errorbar(this_cv_results['param_max_depth'],
             this_cv_results['mean_test_score'],
             yerr=this_cv_results['std_test_score'],
             fmt='o');
plt.xlabel('max depth')
plt.ylabel('accuracy');

In [ ]:
this_cv_results = cv_results[cv_results['param_max_depth'] == 15]
plt.errorbar(this_cv_results['param_n_estimators'],
             this_cv_results['mean_test_score'],
             yerr=this_cv_results['std_test_score'],
             fmt='o');
plt.xlabel('nb estimators')
plt.ylabel('accuracy');

## Now let's automate the search

All wrapper-type hyperparameter optimization will need a function with a parameter that represents the hyperparameter, and which returns a score (usually to be minimized). The following function thus trains a random forest classifier with two hyperparameters, stored in a pair <code>x_int</code>.

In [ ]:
i = 0

def objective(params):
    global i
    i += 1
    print(params)
    clf = RandomForestClassifier(n_estimators=300,
                                 max_depth=int(params['max_depth']),
                                 max_features=params['max_features'])
    scores = cross_val_score(clf, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=3)
    score = np.mean(scores)
    print("SCORE: %s" % score)
    df_result_hyperopt.loc[i, ['score'] + list(params.keys())] = \
        [score] + list(params.values())
    return {'loss': 1. - score, 'status': STATUS_OK}

In [ ]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

space = {
    'max_depth': hp.quniform('max_depth', 2, 30, 1),
    'max_features': hp.quniform('max_features', 0.7, 1, 0.05)
}

df_result_hyperopt = pd.DataFrame(
    columns=['score'] + list(space.keys()))

trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest,
            max_evals=15, trials=trials)

print("Best: %s" % best)

In [ ]:
df_result_hyperopt.sort_values(by='score', ascending=False).head(10)

## Exercises

<ol>
<li> Take any scikit-learn classifier and tune it using hyperopt.
<li> Download any other training set from, e.g., the <a href=http://archive.ics.uci.edu/ml/>UCI Irvine data repository</a>, load it, and optimize any scikit-learn classifier and tune it using pysmac or hyperopt.
<li> Take the <a href="https://drive.google.com/file/d/0B_sb8NJ9KsLUd2pzdjZ3VGxZLWM/view?usp=sharing">notebook</a> that describes the RAMP on variable star classification, assemble a pipeline of a parametrized feature extractor and a classifier, and optimize the full pipeline.
</ol>